# Problema das N-Rainhas – Hill-Climbing

Este notebook resolve o problema das N-Rainhas utilizando o algoritmo **Hill-Climbing**, uma técnica de busca local que tenta melhorar progressivamente uma solução inicial aleatória.

## Objetivo

Colocar N rainhas em um tabuleiro \( N \times N \) de forma que nenhuma ataque a outra, ou seja:
- Nenhuma rainha na mesma linha
- Nenhuma rainha na mesma coluna
- Nenhuma rainha na mesma diagonal


In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import pandas as pd


### Funções auxiliares: avaliação de conflitos e visualização da solução
Define a função `h(state)` que calcula o número de conflitos entre rainhas em um tabuleiro. Também define a função `plot_solution()` para desenhar visualmente a configuração final do tabuleiro.


In [2]:
def conflict(row1, col1, row2, col2):
    return (row1 == row2 or
            col1 == col2 or
            row1 - col1 == row2 - col2 or
            row1 + col1 == row2 + col2)

def h(state):
    """Número total de conflitos entre rainhas"""
    conflicts = 0
    N = len(state)
    for i in range(N):
        for j in range(i + 1, N):
            if conflict(state[i], i, state[j], j):
                conflicts += 1
    return conflicts

def plot_solution(solution, N):
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, N))
    ax.set_ylim((0, N))
    for i, row in enumerate(solution):
        ax.add_patch(patches.Rectangle((i, row), 1, 1, color='black'))
    plt.gca().invert_yaxis()
    plt.show()


### Função principal: Algoritmo Hill-Climbing
Implementa o algoritmo Hill-Climbing com até 5 reinícios aleatórios. A cada reinício, começa com uma solução aleatória e busca vizinhos com menos conflitos, trocando a rainha de linha dentro da mesma coluna.


In [3]:
def hill_climbing(N, max_restarts=5):
    best_overall = None
    best_fitness = float('inf')
    total_time = 0
    results = []

    for run in range(1, max_restarts + 1):
        start = time.time()

        # Estado inicial aleatório: uma rainha em cada coluna
        current = [random.randint(0, N - 1) for _ in range(N)]
        current_fitness = h(current)

        improved = True
        while improved and current_fitness > 0:
            improved = False
            neighbor = current[:]
            best_neighbor = current[:]
            best_neighbor_fitness = current_fitness

            for col in range(N):
                original_row = current[col]
                for row in range(N):
                    if row != original_row:
                        neighbor[col] = row
                        score = h(neighbor)
                        if score < best_neighbor_fitness:
                            best_neighbor_fitness = score
                            best_neighbor = neighbor[:]
                            improved = True
                neighbor[col] = original_row

            current = best_neighbor
            current_fitness = best_neighbor_fitness

        end = time.time()
        solved = current_fitness == 0
        exec_time = end - start

        results.append({
            "N": N,
            "Run": run,
            "Solved": solved,
            "Fitness": current_fitness,
            "Time": exec_time,
            "Solution": current
        })

        if current_fitness < best_fitness:
            best_overall = current[:]
            best_fitness = current_fitness
            total_time = exec_time

    return results, best_overall, best_fitness, total_time


### Execução do algoritmo para diferentes tamanhos de tabuleiro
Executa o algoritmo Hill-Climbing para N = 32, 64 e 128. Para cada valor de N, roda o algoritmo 5 vezes e exibe os resultados. Também plota a melhor solução encontrada (se livre de conflitos).


In [7]:
all_results = []

for N in [32, 64, 128]:
    print(f"\nHill-Climbing para N={N}")
    results, best_sol, best_fit, total_time = hill_climbing(N, max_restarts=5)

    for r in results:
        print(f"Run {r['Run']}: Solved={r['Solved']} | Fitness={r['Fitness']} | Time={r['Time']:.2f}s")
        all_results.append(r)

    if best_fit == 0:
        print("➡️ Solução encontrada:")
        plot_solution(best_sol, N)
    else:
        print(f"⚠️ Melhor solução tinha {best_fit} conflitos")



Hill-Climbing para N=32
Run 1: Solved=False | Fitness=5 | Time=1.81s
Run 2: Solved=False | Fitness=3 | Time=3.41s
Run 3: Solved=False | Fitness=4 | Time=3.03s
Run 4: Solved=False | Fitness=1 | Time=3.55s
Run 5: Solved=False | Fitness=2 | Time=3.34s
⚠️ Melhor solução tinha 1 conflitos

Hill-Climbing para N=64
Run 1: Solved=False | Fitness=3 | Time=71.88s
Run 2: Solved=False | Fitness=6 | Time=52.62s
Run 3: Solved=False | Fitness=2 | Time=55.20s
Run 4: Solved=False | Fitness=2 | Time=53.90s
Run 5: Solved=False | Fitness=3 | Time=59.80s
⚠️ Melhor solução tinha 2 conflitos

Hill-Climbing para N=128
Run 1: Solved=False | Fitness=6 | Time=1435.91s
Run 2: Solved=False | Fitness=4 | Time=1580.34s
Run 3: Solved=False | Fitness=5 | Time=1446.04s
Run 4: Solved=False | Fitness=4 | Time=1697.27s
Run 5: Solved=False | Fitness=6 | Time=2422.67s
⚠️ Melhor solução tinha 4 conflitos


In [8]:
df_hc = pd.DataFrame([{
    "N": r["N"],
    "Run": r["Run"],
    "Solved": r["Solved"],
    "Fitness": r["Fitness"],
    "Time (s)": r["Time"]
} for r in all_results])

df_hc


,N,Run,Solved,Fitness,Time (s)
0,32,1,False,5,1.806693
1,32,2,False,3,3.407090
2,32,3,False,4,3.028082
3,32,4,False,1,3.545154
4,32,5,False,2,3.337911
5,64,1,False,3,71.876492
6,64,2,False,6,52.617342
7,64,3,False,2,55.200180
8,64,4,False,2,53.895159
9,64,5,False,3,59.801607
